In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.models import squeezenet1_0, resnet18
from google.colab import drive
import os
import random
import numpy as np

drive.mount('/content/drive')
device = 'cuda' if torch.cuda.is_available() else 'cpu'

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if device == 'cuda':
        torch.cuda.manual_seed(seed)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
!pip install torchattack

In [10]:
from torchattack import PGD

In [11]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3)
        self.conv2 = nn.Conv2d(16, 32, 3)
        self.fc1 = nn.Linear(32 * 5 * 5, 10)
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = x.view(x.size(0), -1)
        return self.fc1(x)

class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5); self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(256, 120); self.fc2 = nn.Linear(120, 84); self.fc3 = nn.Linear(84, 10)
    def forward(self, x):
        x = F.relu(self.conv1(x)); x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x)); x = F.max_pool2d(x, 2)
        x = x.view(x.size(0), -1); x = F.relu(self.fc1(x)); x = F.relu(self.fc2(x)); return self.fc3(x)

class SqueezeNetMNIST(nn.Module):
    def __init__(self):
        super(SqueezeNetMNIST, self).__init__()
        self.model = squeezenet1_0(num_classes=10)
        self.model.classifier[1] = nn.Conv2d(512, 10, kernel_size=1)
    def forward(self, x):
        if x.shape[1] == 1: x = x.repeat(1, 3, 1, 1)
        return self.model(x)

class ResNet18MNIST(nn.Module):
    def __init__(self):
        super(ResNet18MNIST, self).__init__()
        self.model = resnet18(num_classes=10)
        self.model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
    def forward(self, x):
        return self.model(x)

class LinearModel(nn.Module):
    def __init__(self):
        super(LinearModel, self).__init__()
        self.flatten = nn.Flatten()
        self.linear = nn.Linear(28*28, 10)
    def forward(self, x):
        x = self.flatten(x)
        return self.linear(x)

class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28*28, 256)
        self.fc2 = nn.Linear(256, 10)
    def forward(self, x):
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

class ConvNetTiny(nn.Module):
    def __init__(self):
        super(ConvNetTiny, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3)
        self.fc1 = nn.Linear(8 * 13 * 13, 10)
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = x.view(x.size(0), -1)
        return self.fc1(x)

class ConvNetWide(nn.Module):
    def __init__(self):
        super(ConvNetWide, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.fc1 = nn.Linear(64 * 5 * 5, 10)
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = x.view(x.size(0), -1)
        return self.fc1(x)

class ConvNetDeep(nn.Module):
    def __init__(self):
        super(ConvNetDeep, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1), nn.ReLU(),
            nn.Conv2d(16, 16, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, padding=1), nn.ReLU(),
            nn.Conv2d(32, 32, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Linear(32 * 7 * 7, 10)
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)

class MiniVGG(nn.Module):
    def __init__(self):
        super(MiniVGG, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 64, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Linear(64 * 3 * 3, 10)
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)

class InceptionModule(nn.Module):
    def __init__(self, in_channels, out_1x1, red_3x3, out_3x3, red_5x5, out_5x5, out_pool):
        super(InceptionModule, self).__init__()
        self.branch1 = nn.Conv2d(in_channels, out_1x1, kernel_size=1)
        self.branch2 = nn.Sequential(
            nn.Conv2d(in_channels, red_3x3, kernel_size=1),
            nn.Conv2d(red_3x3, out_3x3, kernel_size=3, padding=1)
        )
        self.branch3 = nn.Sequential(
            nn.Conv2d(in_channels, red_5x5, kernel_size=1),
            nn.Conv2d(red_5x5, out_5x5, kernel_size=5, padding=2)
        )
        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_channels, out_pool, kernel_size=1)
        )
    def forward(self, x):
        return torch.cat([self.branch1(x), self.branch2(x), self.branch3(x), self.branch4(x)], 1)

class MiniGoogLeNet(nn.Module):
    def __init__(self):
        super(MiniGoogLeNet, self).__init__()
        self.pre_layers = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.inception = InceptionModule(64, 16, 32, 24, 8, 8, 16)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(64, 10)
    def forward(self, x):
        x = self.pre_layers(x)
        x = self.inception(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

class MiniDenseNet(nn.Module):
    def __init__(self):
        super(MiniDenseNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 16, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 16, 3, padding=1)
        self.pool = nn.MaxPool2d(2)
        self.fc = nn.Linear(16 * 14 * 14, 10)
    def forward(self, x):
        out1 = F.relu(self.conv1(x))
        out2 = F.relu(self.conv2(out1))
        c2 = torch.cat([out1, out2], 1)
        out3 = F.relu(self.conv3(c2))
        x = self.pool(out3)
        x = x.view(x.size(0), -1)
        return self.fc(x)

base_path = '/content/drive/My Drive/adversarial_mnist'
weights_dir = f'{base_path}/model_weights'
adv_dir = f'{base_path}/large_adversarial_examples'
os.makedirs(weights_dir, exist_ok=True)
os.makedirs(adv_dir, exist_ok=True)

transform = transforms.Compose([transforms.ToTensor()])
train_set = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_set = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
train_loader = DataLoader(train_set, batch_size=128, shuffle=True)
test_loader = DataLoader(test_set, batch_size=1000, shuffle=False)

In [12]:
model_configs = []
architectures = [
    ('Linear', LinearModel),
    ('MLP', MLP),
    ('ConvTiny', ConvNetTiny),
    ('Simple', SimpleCNN),
    ('ConvWide', ConvNetWide),
    ('ConvDeep', ConvNetDeep),
    ('LeNet', LeNet5),
    ('MiniVGG', MiniVGG),
    ('Squeeze', SqueezeNetMNIST),
    ('MiniInception', MiniGoogLeNet),
    ('MiniDense', MiniDenseNet),
    ('ResNet', ResNet18MNIST)
]

for seed in range(2):
    for name, arch_cls in architectures:
        model_name = f"{name}_s{seed}"
        model_file = f"{name.lower()}_s{seed}.pth"
        model_configs.append((model_name, model_file, arch_cls, seed))

print(f"Total models in zoo: {len(model_configs)}")

Total models in zoo: 24


In [14]:
def train_model(model, loader, epochs=10):
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()
    model.train()
    for epoch in range(epochs):
        for data, target in loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

for name, fname, arch_cls, seed in model_configs:
    path = f"{weights_dir}/{fname}"
    print(f"Training {name}...")
    set_seed(seed)
    model = arch_cls().to(device)
    train_model(model, train_loader)
    torch.save(model.state_dict(), path)
    print(f"Saved {name}")

Training Linear_s0...
Saved Linear_s0
Training MLP_s0...
Saved MLP_s0
Training ConvTiny_s0...
Saved ConvTiny_s0
Training Simple_s0...
Saved Simple_s0
Training ConvWide_s0...
Saved ConvWide_s0
Training ConvDeep_s0...
Saved ConvDeep_s0
Training LeNet_s0...
Saved LeNet_s0
Training MiniVGG_s0...
Saved MiniVGG_s0
Training Squeeze_s0...
Saved Squeeze_s0
Training MiniInception_s0...
Saved MiniInception_s0
Training MiniDense_s0...
Saved MiniDense_s0
Training ResNet_s0...
Saved ResNet_s0
Training Linear_s1...
Saved Linear_s1
Training MLP_s1...
Saved MLP_s1
Training ConvTiny_s1...
Saved ConvTiny_s1
Training Simple_s1...
Saved Simple_s1
Training ConvWide_s1...
Saved ConvWide_s1
Training ConvDeep_s1...
Saved ConvDeep_s1
Training LeNet_s1...
Saved LeNet_s1
Training MiniVGG_s1...
Saved MiniVGG_s1
Training Squeeze_s1...
Saved Squeeze_s1
Training MiniInception_s1...
Saved MiniInception_s1
Training MiniDense_s1...
Saved MiniDense_s1
Training ResNet_s1...
Saved ResNet_s1


In [15]:
def evaluate_clean(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1)
            correct += (pred == target).sum().item()
            total += target.size(0)
    return 100 * correct / total

for name, fname, arch_cls, seed in model_configs:
    path = f"{weights_dir}/{fname}"
    save_path = f"{adv_dir}/500_adv_{fname}"

    print(f"Generating adversarial data for {name}...")
    model = arch_cls().to(device)
    model.load_state_dict(torch.load(path, map_location=device))
    model.eval()

    acc = evaluate_clean(model, test_loader)

    attack = PGD(model, eps=0.3, alpha=0.01, steps=40, random_start=True)

    adv_images = []
    clean_images = []
    labels = []

    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        if len(adv_images) * test_loader.batch_size >= 500:
            break

        adv = attack(data, target)
        adv_images.append(adv.cpu())
        clean_images.append(data.cpu())
        labels.append(target.cpu())

    adv_images = torch.cat(adv_images)[:500]
    clean_images = torch.cat(clean_images)[:500]
    labels = torch.cat(labels)[:500]

    with torch.no_grad():
        preds = model(adv_images.to(device)).argmax(dim=1)
        rob_acc = (preds == labels.to(device)).float().mean().item() * 100

    torch.save({
        'clean': clean_images,
        'adv': adv_images,
        'lbl': labels,
        'score_clean': acc,
        'score_robust': rob_acc
    }, save_path)
    print(f"Saved dataset for {name} (Clean: {acc:.1f}%, Robust: {rob_acc:.1f}%) ")

Generating adversarial data for Linear_s0...
Saved dataset for Linear_s0 (Clean: 92.7%, Robust: 0.0%) 
Generating adversarial data for MLP_s0...
Saved dataset for MLP_s0 (Clean: 97.9%, Robust: 0.0%) 
Generating adversarial data for ConvTiny_s0...
Saved dataset for ConvTiny_s0 (Clean: 97.5%, Robust: 0.0%) 
Generating adversarial data for Simple_s0...
Saved dataset for Simple_s0 (Clean: 98.8%, Robust: 0.0%) 
Generating adversarial data for ConvWide_s0...
Saved dataset for ConvWide_s0 (Clean: 99.0%, Robust: 0.0%) 
Generating adversarial data for ConvDeep_s0...
Saved dataset for ConvDeep_s0 (Clean: 99.2%, Robust: 0.0%) 
Generating adversarial data for LeNet_s0...
Saved dataset for LeNet_s0 (Clean: 98.8%, Robust: 0.0%) 
Generating adversarial data for MiniVGG_s0...
Saved dataset for MiniVGG_s0 (Clean: 99.2%, Robust: 0.0%) 
Generating adversarial data for Squeeze_s0...
Saved dataset for Squeeze_s0 (Clean: 88.5%, Robust: 9.8%) 
Generating adversarial data for MiniInception_s0...
Saved dataset